# Spatial operations - Python

## GeoParquet format

The `obistherm` dataset is saved on GeoParquet format. Thus, it contains a geometry column in WKB format. On Python, you can directly read the dataset using the library `geopandas`.

_Note: to see how to download the data, check [this other notebook.](https://github.com/iobis/obis-therm/blob/main/notebooks/data_access_py.ipynb)_

In [ ]:
import geopandas

local_folder = "aggregated"
filters = [("species", "==", "Leptuca thayeri")]
gdf = geopandas.read_parquet(local_folder, filters=filters)[["species", "coraltempSST", "geometry"]]
gdf

Note that, differently from the other operations, this time we did the filter directly on the `read_parquet` function from `geopandas`.

We can do very quick spatial operations with this `geopandas` object. For example, crop to a certain area and plot.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

gdf_cropped = gdf.cx[-70:-30, -25:29]

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

ax.add_feature(cfeature.LAND, facecolor='gray')

gdf_cropped.plot(ax=ax, column="coraltempSST", cmap="coolwarm", legend=True, markersize=24, alpha=0.9, transform=ccrs.PlateCarree())

ax.set_xlim(-50, -30)
ax.set_ylim(-25, 0)

ax.set_title("Spatial Distribution of Leptuca thayeri with CoralTemp SST", fontsize=14)

plt.show()

## H3 grid system

The `obistherm` dataset also comes with pre-computed H3 index at resolution 7. The H3 grid system is a hierarchical, hexagon-based spatial indexing system developed by Uber that divides the globe into hexagonal cells at multiple resolutions, allowing for efficient spatial analysis and indexing. It enables seamless geospatial operations like clustering, nearest-neighbor queries, and region aggregation with consistent cell areas and shapes across the globe. You can learn more about it [here](https://h3geo.org/).

You can do filtering and aggregation efficiently using this system. For example, let's consider a certain area in Europe:

In [ ]:
import h3

polygon_wkt = "POLYGON ((2.548828 53.448807, -0.109863 51.57707, 2.416992 50.764259, 7.646484 53.225768, 2.548828 53.448807))"
europe_area = geopandas.GeoSeries.from_wkt([polygon_wkt], crs="EPSG:4326")

geojson_polygon = europe_area.geometry[0].__geo_interface__

resolution = 7
europe_h3 = h3.polyfill(geojson_polygon, resolution)
filters = [("h3_7", "in", europe_h3)]

data_area = geopandas.read_parquet(local_folder, filters=filters)[["species", "family", "year", "coraltempSST", "h3_7"]]

data_europe = data_area[data_area['family'] == "Laridae"]
data_europe

The first part (selecting the data for a certain region) we could have done using the spatial capabilities of `geopandas`, but H3 will enable us to aggregate the data very easily now.

Let's first go to a coarser resolution, 6, get the average of temperatures in each site and see how much it deviates from the average of all records.

In [ ]:
Laridae_average = data_europe['coraltempSST'].mean()

data_europe['h3_6'] = data_europe['h3_7'].apply(lambda h3_index: h3.h3_to_parent(h3_index, 6))

# Step 3: Group by the coarser H3 grid and calculate the temperature difference
Laridae_diff = data_europe.groupby('h3_6').agg(
    deep_dif=('coraltempSST', lambda x: x.mean() - Laridae_average)
).reset_index()

print(Laridae_diff.head())

We can then plot the result. We will convert the H3 codes to a hexagonal grid for better plotting:

In [45]:
import shapely.geometry as geom

def h3_to_polygon(h3_index):
    boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    return geom.Polygon(boundary)

Laridae_diff['geometry'] = Laridae_diff['h3_6'].apply(h3_to_polygon)

gdf_Laridae_diff = geopandas.GeoDataFrame(Laridae_diff, geometry='geometry', crs="EPSG:4326")


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

ax.add_feature(cfeature.LAND, facecolor='gray')

gdf_Laridae_diff.plot(ax=ax, column='deep_dif', cmap='PuOr', legend=True, edgecolor='black', alpha=0.7)
 
ax.set_title("Laridae Deep Temperature Differences", fontsize=14)

plt.show()
